# CNNfor MNIST Dataset

### 패키지

In [12]:
import numpy as np
import math

import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import losses
from keras import backend as K

from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

from sklearn.manifold import TSNE
from skimage import io

import theano

import json

In [13]:
 # Functions 
def get_hot_idx(arr):
    return arr.index(max(arr))

def save_to_json_file(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)
    print(filename + ' 저장완료')
    
def get_round_array(array, decimal):
    return [round(e, decimal) for e in array]

def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
    activations = get_activations([X_batch,0])
    return activations

def get_arr_from_json_file(filename):
    input_file = open (filename)
    return json.load(input_file)
    
current_milli_time = lambda: int(round(time.time() * 1000))

### 데이터셋 로드

In [17]:
# 데이터 관련 파라메터 정의
img_rows, img_cols = 28, 28
num_of_feature = img_rows * img_cols 

num_of_class = 10
num_of_trainset = 10000
num_of_testset = 10000

# the data, split between train and test sets
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)[:num_of_trainset]
Y_tarin = Y_train[:num_of_trainset]
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)[:num_of_testset]
Y_test = Y_test[:num_of_testset]

input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices
Y_train = keras.utils.to_categorical(Y_train, num_of_class)
Y_test = keras.utils.to_categorical(Y_test, num_of_class)

In [24]:
# 로컬 테스트 데이터 로드
images = np.zeros((10000, 784))
for real in range(10):
    for idx in range(1, 1001):
        file = '../../data/mnist/images/'+ str(real) + '/' + str(real) + '_' + str(idx) +'.png'
        image = np.ndarray.flatten(io.imread(file)) / 255.0
        image = np.array([1 - pixel for pixel in image])
        images[real * 1000 + idx - 1] = image

images = images.reshape(images.shape[0], img_rows, img_cols, 1)

### 모델 구성

In [38]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(num_of_class, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### 모델 학습

In [39]:
model.fit(X_train, Y_train[:num_of_trainset],
          epochs=3,
          batch_size=32,
          verbose=1,
          validation_data=(X_test, Y_test))

Train on 10000 samples, validate on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 3s 277us/step - loss: 0.4720 - acc: 0.8618 - val_loss: 0.2696 - val_acc: 0.9199
Epoch 2/3
10000/10000 [==============================] - 3s 250us/step - loss: 0.2136 - acc: 0.9399 - val_loss: 0.1930 - val_acc: 0.9447
Epoch 3/3
10000/10000 [==============================] - 3s 257us/step - loss: 0.1569 - acc: 0.9558 - val_loss: 0.1534 - val_acc: 0.9565


### 테스트 셋 결과 확인

In [40]:
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)
print('loss_and_metrics : ' + str(loss_and_metrics))

10000/10000 [==============================] - 1s 56us/step
loss_and_metrics : [0.1534139785528183, 0.9565]


## 내 데이터로 테스트

In [41]:
# 예측
pred_proba =  model.predict_proba(images).tolist()

In [42]:
# 성능 확인 및 저장
correct = 0
performances = {
    "accuracy": 0,
    "recall": [0] * 10,
    "precision": [0] * 10
}
truePredict = [0] * 10
numOfPredict = [0] * 10
predicts = []

for i in range(len(pred_proba)):
    prob = pred_proba[i]
    pred = get_hot_idx(prob)
    real = i // 1000
    predicts.append(dict({
        "real": real,
        "pred": pred,
        "prob": [round(e, 4) for e in prob]
    }))
    numOfPredict[pred] = numOfPredict[pred] + 1
    if pred is real:
        truePredict[real] = truePredict[real] + 1
        correct = correct + 1

performances["accuracy"] = correct / 10000
performances["recall"] = [round(truePredict[i] / 1000, 4) for i in range(10) ]
performances["precision"] = [round(truePredict[i] / numOfPredict[i], 4) for i in range(10)]

print(performances)

{'accuracy': 0.9494, 'recall': [0.968, 0.961, 0.956, 0.963, 0.977, 0.914, 0.973, 0.945, 0.914, 0.923], 'precision': [0.968, 0.9806, 0.9087, 0.9198, 0.9403, 0.9775, 0.9586, 0.9394, 0.9551, 0.9525]}


### 차원축소

1) 마지막 직전의 레이어에서 activation values를 뽑는다.

2) t-SNE로 차원을 축소한다.

In [32]:
samples = []
idxs = get_arr_from_json_file('./sample_image_idxs.json')
for idx in idxs:
    samples.append(images[idx])
print(len(samples))

500


In [33]:
extracted_features = get_activations(model, -2, samples)[0]
y = TSNE(n_components=2).fit_transform(extracted_features)
sne_map = []
for e in y:
    sne_map.append({
        "x": round(e[0], 2),
        "y": round(e[1], 2)
    })

In [34]:
model_data = {
    "model_name": 'Convolutional Neural Network',
    "short_name": 'CNN',
    "description": '1개의 Convolution2D 레이어와 1개의 Dense 레이어를 가지는 모델이다.',
    "performance": {
        "accuracy": performances["accuracy"],
        "recall": performances["recall"],
        "precision": performances["precision"]
    },
    "predict": predicts,
    "t-sne": sne_map
}

In [35]:
with open('mnist_cnn.json', 'w', encoding='utf-8') as f:
    json.dump(str(model_data), f, ensure_ascii=False, indent=4)